# Integration and Visualization
- YOUR NAME:
- YOUR VUNetID:

(If you do not provide your NAME and VUNetID we will not accept your submission.)

**To hand in: one zip file, containing...**
- this file, with completed tasks
- the ontology you made in the previous assignment (Task 1)
- an ontology you found on the web (Task 1)
- an integrated ontology (`mapping.ttl`) (Task 1)

## Task 1 (20 Points):  Integrating ontologies in Protégé
Take the ontology you built for Module 4 (let’s call it ontology A), and find either a fellow student with a similar ontology or an ontology on the web on similar domain. That will be our ontology B. 

Create an empty ontology in Protégé (ontology C), and import both ontologies. 

Define at least 6 mappings as following:

-    1 mapping between a class from A and a class from B using rdfs:subClassOf 
-    1 mapping between a class from A and a class from B using owl:equivalentClass
-    1 mapping between a property from A and a property from B using rdfs:subPropertyOf
-    1 mapping between a property from A and a property from B using either owl:equivalentProperty, owl:disjointProperty, owl:inverseOf, or owl:propertyChainAxiom
-    1 mapping between an individual from A and an individual from B using owl:differentFrom
-    1 mapping between an individual from A and an individual from B using owl:sameAs 

The combined ontology should be **consistent**.

Save the ontology as Turtle, where the filename is ‘`mapping.ttl`’

**Submit all three ontologies together with this assignment, leaving preserved the original file names of the ontologies**

In [ ]:
print(open('mapping.ttl').read())

## Task 2 (10 Points):  Explain your mapping

Please explain and motivate the mapping you made in the previous question in a few sentences in the text field below :

...

## Task 3 (20 points). From SPARQL to DataFrame

For the next few tasks we are going to store the results from our queries in a [Pandas DataFrame](https://pandas.pydata.org), making it easier to perform further data processing on the results. To accomplish this, write a procedure to execute the query and to convert the results into a dataframe. 

To help you on your way, we have already written the main procedure `table_from_query/2` and the helper function `cast/1` which converts the raw values to the appropriate Python objects. Your task is to write the two missing procedures: `execute_query/2` and `create_dataframe/1`. *HINT: revisit assignment 3 if you are unsure where to start*.

Please run the next two cells to import the necessary dependencies and to activate the helper procedures.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [ ]:
def cast(value):
    """ Cast the value from a SPARQL result to an appropriate Python object.
        The expected input is a dictionary with at least the keys 'type' and 'value'.
    """
    v = value['value']
    if value['type'] in ['literal', 'typed-literal'] and 'datatype' in value.keys():
        dtype = URIRef(value['datatype'])
        if any(d in dtype for d in ('integer', 'long', 'int', 'short', 'byte')):
            return int(v)
        if any(d in dtype for d in ('decimal', 'float', 'double')):
            return float(v)

    # fallback to string
    return str(v)

def table_from_query(endpoint, query):
    """ Execute a query on an endpoint and return the results as a dataframe.
    """
    query_results = execute_query(endpoint, query)
    dataframe = create_dataframe(query_results)

    return dataframe

In [ ]:
def execute_query():
    # your code here

def create_dataframe():
    # your code here

To test your code, we ask you to create a dataframe about teachers and their course load. We will use the data in `vuDataset.ttl`, which must be imported in GraphDB as a new repository called `repo-vu`. Use the following endpoint and query for this purpose.


In [ ]:
sparql_endpoint = "http://localhost:7200/repositories/repo-vu"
sparql_query = """
PREFIX vu: <http://example.org/vu/>

SELECT ?teacher (COUNT(*) as ?courses) {
    ?teacher vu:teaches ?o .
} GROUP BY ?teacher
"""

courseload = table_from_query(sparql_endpoint, sparql_query)
print(courseload)

## Task 4 (20 points) - Data visualization

Visualizing (parts of) the data can help us gain a better understanding of what we are dealing with. Now that the data is stored as a dataframe, we can use a library like `matplotlib` to create a variety of helpful plots. To illustrate this, run the following cells to install and import the library, and to generate a pie plot from our dataframe about course loads.

In [ ]:
%pip install pandas matplotlib

In [ ]:
import matplotlib

In [ ]:
# Pandas visualization
courseload.set_index('teacher', inplace=True)
courseload['courses'].plot.pie()

## Task 4a (10 points): Make a horizontal bar chart (with suitable labels and title) from the `courseload` dataframe

There are many different kinds of plots that we can make. Please look at [the Pandas wiki](https://pandas.pydata.org/docs/user_guide/visualization.html) to get an idea on what is possible.

In [ ]:
# your code here

## Task 4b (10 points): Write a new SPARQL query for DBpedia and create an interesting chart from it

In [ ]:
# your code here

## Task 5 (10 points). Conditional styling

Below is a query to find all EU countries and their surface area in $m^2$. Below that is some code that adds some colour to the resulting table.


We ask you to update the query and code below to add a new column that shows the percentage of the country's area that is water.
The background color of this column must depend on this percentage: it should be greener if more of the country is land, and bluer if more of the country is water.

In [ ]:
endpoint = "http://dbpedia.org/sparql"
query = """
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX yago: <http://dbpedia.org/class/yago/>

SELECT DISTINCT ?name ?area_100k_km2 ?giniCoefficient 
WHERE { 
    ?country 
        a yago:WikicatMemberStatesOfTheEuropeanUnion ;
        dbo:area ?area ; # area is stated in m^2
        dbo:giniCoefficient ?giniCoefficient ;
        rdfs:label ?name. 
    
    BIND (ROUND(?area/100000000000) AS ?area_100k_km2)
    FILTER(LANG(?name) = 'en')
}
ORDER BY DESC(?area_100k_km2)
LIMIT 50"""

dbpedia_countries = table_from_query(endpoint, query)

In [ ]:
# Styling: https://pandas.pydata.org/docs/user_guide/style.html
# Using CSS Hue-Saturation-Level colors: https://www.w3schools.com/colors/colors_hsl.asp

def gini_color(gini):
    return f'background: hsl({100 - gini}, 100%, 50%)' 

dbpedia_countries.style \
    .map(gini_color, subset=(slice(None), "giniCoefficient"))

# Map Visualizations

Below is a query to find all EU capitals, together with their coordinates. The results are drawn on a map using the library `folium`. 

First run the following cells to install and import the library.

In [ ]:
%pip install folium

In [ ]:
import folium

# Task 6 (20 points) - Population counts

We ask you to update the query and code below to make the markers (circles) scale with the population of the capital (but keep it readable!).

In [ ]:
endpoint = "http://query.wikidata.org/sparql"
query = """
SELECT ?capitalLabel ?coords 
WHERE {
    ?country
        wdt:P463 wd:Q458 ; # member of: European Union
        wdt:P36 ?capital . # capital: ?capital
        
    ?capital 
        wdt:P625 ?coords . # coordinate location: ?coords
    
    # Using Wikidata-only label service
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""

capitals = table_from_query(endpoint, query)

In [ ]:
# Extract latitude and longitude from coordinates in Well-Known Text (WKT) format
capitals_coords = capitals.join(capitals.coords.str.extract('Point\\((?P<lon>\\S+) (?P<lat>\\S+)\\)').astype(float))

middle = [capitals_coords['lat'].mean(), capitals_coords['lon'].mean()]
m = folium.Map(location=middle, zoom_start=4)
for _, row in capitals_coords.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius= 20,
        popup= row['capitalLabel'],
        color="#3186cc",
        fill=True,
        fill_color="#3186cc",
    ).add_to(m)

m